In [1]:
# imports
import requests
import pandas as pd
import os
import json

In [2]:
yelp_apikey = os.environ["yelp_api"]

In [51]:
def get_yelp_ids():
    """
    Retrieves and returns the list of networks.
    """
    request_url = 'https://api.yelp.com/v3/businesses/search?location=Austin%2C%20TX%2078701&radius=1000&categories=bar&categories=entertainment&attributes=&sort_by=best_match&limit=25'
    headers = {
    "accept": "application/json",
    "Authorization": os.environ["yelp_api"]
    }
    response = requests.get(request_url, headers=headers)
    return response.json()


In [116]:
# Call the function to retrieve the Yelp IDs and names
yelp_data = get_yelp_ids()

# Create a DataFrame from the businesses data
df = pd.DataFrame(yelp_data['businesses'])

# Extract the 'id' column into a separate DataFrame
id_df = df[['id', 'name','categories', 'review_count', 'rating', 'coordinates', 'location']]

id_df = id_df.rename(columns={'id':'yelp_id'})

# Print the id_df DataFrame
id_df.head()

,yelp_id,name,categories,review_count,rating,coordinates,location
0,cs6HfZNykLVitm09jWFqWg,Moonshine Patio Bar & Grill,"[{'alias': 'southern', 'title': 'Southern'}, {...",5778,4.5,"{'latitude': 30.26375388334444, 'longitude': -...","{'address1': '303 Red River St', 'address2': '..."
1,XOo0oa5sXCZGjKXapIN95w,Red Ash,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",1209,4.5,"{'latitude': 30.26588, 'longitude': -97.744672}","{'address1': '303 Colorado St', 'address2': No..."
2,Tta8AYBZq3fwr1n2up75vQ,Eureka!,"[{'alias': 'tradamerican', 'title': 'American ...",1065,4.0,"{'latitude': 30.2678, 'longitude': -97.7412}","{'address1': '200 E 6th St', 'address2': '', '..."
3,CqOkl94q9ttvYNWIITA8ow,Swift's Attic,"[{'alias': 'newamerican', 'title': 'American (...",1166,4.0,"{'latitude': 30.26563, 'longitude': -97.74337}","{'address1': '315 Congress Ave', 'address2': '..."
4,v1UzkU8lEWdjxq8byWFOKg,Gus's World Famous Fried Chicken - Austin,"[{'alias': 'southern', 'title': 'Southern'}, {...",3101,4.5,"{'latitude': 30.2634862, 'longitude': -97.7417...","{'address1': '117 San Jacinto Blvd', 'address2..."


In [104]:
category_df = pd.json_normalize(id_df['categories'])
category_df = category_df.rename(columns={0:'cat_types'})
category_df = category_df.drop(columns=[1,2])
category_df.head()

,cat_types
0,"{'alias': 'southern', 'title': 'Southern'}"
1,"{'alias': 'italian', 'title': 'Italian'}"
2,"{'alias': 'tradamerican', 'title': 'American (..."
3,"{'alias': 'newamerican', 'title': 'American (N..."
4,"{'alias': 'southern', 'title': 'Southern'}"


In [105]:
cat_type = pd.json_normalize(category_df['cat_types'])
cat_type = cat_type.rename(columns={'title':'category'})
cat_type = cat_type.drop(columns=['alias'])
cat_type.head()

,category
0,Southern
1,Italian
2,American (Traditional)
3,American (New)
4,Southern


In [117]:
merge_cat = pd.concat([category_df, cat_type],axis=1)
merge_cat = merge_cat.drop(columns=['cat_types'])
merge_cat.head()

,category
0,Southern
1,Italian
2,American (Traditional)
3,American (New)
4,Southern


In [123]:
# Extract additional columns
additional_cols = df[['review_count', 'rating', 'coordinates', 'location']]

# Concatenate all DataFrames
add_cat = pd.concat([id_df[['yelp_id', 'name']], merge_cat, additional_cols], axis=1)
add_cat.head()

,yelp_id,name,category,review_count,rating,coordinates,location
0,cs6HfZNykLVitm09jWFqWg,Moonshine Patio Bar & Grill,Southern,5778,4.5,"{'latitude': 30.26375388334444, 'longitude': -...","{'address1': '303 Red River St', 'address2': '..."
1,XOo0oa5sXCZGjKXapIN95w,Red Ash,Italian,1209,4.5,"{'latitude': 30.26588, 'longitude': -97.744672}","{'address1': '303 Colorado St', 'address2': No..."
2,Tta8AYBZq3fwr1n2up75vQ,Eureka!,American (Traditional),1065,4.0,"{'latitude': 30.2678, 'longitude': -97.7412}","{'address1': '200 E 6th St', 'address2': '', '..."
3,CqOkl94q9ttvYNWIITA8ow,Swift's Attic,American (New),1166,4.0,"{'latitude': 30.26563, 'longitude': -97.74337}","{'address1': '315 Congress Ave', 'address2': '..."
4,v1UzkU8lEWdjxq8byWFOKg,Gus's World Famous Fried Chicken - Austin,Southern,3101,4.5,"{'latitude': 30.2634862, 'longitude': -97.7417...","{'address1': '117 San Jacinto Blvd', 'address2..."


In [126]:
coords = pd.json_normalize(add_cat['coordinates'])
coords.head()

,latitude,longitude
0,30.263754,-97.738077
1,30.265880,-97.744672
2,30.267800,-97.741200
3,30.265630,-97.743370
4,30.263486,-97.741731


In [128]:
# Extract additional columns
additional_col = df[['location']]

# Concatenate all DataFrames
add_coords = pd.concat([add_cat[['yelp_id', 'name', 'category', 'review_count', 'rating']], coords, additional_col], axis=1)
add_coords.head()

,yelp_id,name,category,review_count,rating,latitude,longitude,location
0,cs6HfZNykLVitm09jWFqWg,Moonshine Patio Bar & Grill,Southern,5778,4.5,30.263754,-97.738077,"{'address1': '303 Red River St', 'address2': '..."
1,XOo0oa5sXCZGjKXapIN95w,Red Ash,Italian,1209,4.5,30.265880,-97.744672,"{'address1': '303 Colorado St', 'address2': No..."
2,Tta8AYBZq3fwr1n2up75vQ,Eureka!,American (Traditional),1065,4.0,30.267800,-97.741200,"{'address1': '200 E 6th St', 'address2': '', '..."
3,CqOkl94q9ttvYNWIITA8ow,Swift's Attic,American (New),1166,4.0,30.265630,-97.743370,"{'address1': '315 Congress Ave', 'address2': '..."
4,v1UzkU8lEWdjxq8byWFOKg,Gus's World Famous Fried Chicken - Austin,Southern,3101,4.5,30.263486,-97.741731,"{'address1': '117 San Jacinto Blvd', 'address2..."


In [136]:
zip_code = pd.json_normalize(add_coords['location'])
zip_code = zip_code.drop(columns=['address1', 'address2', 'address3', 'city', 'country', 'state', 'display_address'])
zip_code.head()

,zip_code
0,78701
1,78701
2,78701
3,78701
4,78701


In [137]:
# Concatenate all DataFrames
add_loc = pd.concat([add_coords[['yelp_id', 'name', 'category', 'review_count', 'rating', 'latitude', 'longitude']], zip_code], axis=1)
add_loc.head()

,yelp_id,name,category,review_count,rating,latitude,longitude,zip_code
0,cs6HfZNykLVitm09jWFqWg,Moonshine Patio Bar & Grill,Southern,5778,4.5,30.263754,-97.738077,78701
1,XOo0oa5sXCZGjKXapIN95w,Red Ash,Italian,1209,4.5,30.265880,-97.744672,78701
2,Tta8AYBZq3fwr1n2up75vQ,Eureka!,American (Traditional),1065,4.0,30.267800,-97.741200,78701
3,CqOkl94q9ttvYNWIITA8ow,Swift's Attic,American (New),1166,4.0,30.265630,-97.743370,78701
4,v1UzkU8lEWdjxq8byWFOKg,Gus's World Famous Fried Chicken - Austin,Southern,3101,4.5,30.263486,-97.741731,78701


In [138]:
add_loc.to_csv('additional_poi.csv', index=False)